<a href="https://colab.research.google.com/github/Zanderl1987/QuantConnect-Projects/blob/master/QC_Straddle_Options_Strategy_06222020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Version 1.0

In [ ]:
from Execution.ImmediateExecutionModel import ImmediateExecutionModel
#from OptionsUniverseSelectionModel import OptionsUniverseSelectionModel
from datetime import timedelta
import pandas as pd
import numpy as np


class BullCallSpreadAlgorithm(QCAlgorithm):

    def Initialize(self):
        self.SetStartDate(2013, 1, 1)
        self.SetEndDate(2020, 6, 18)
        self.SetCash(200000)
        equity = self.AddEquity("TSLA", Resolution.Minute)
        option = self.AddOption("TSLA", Resolution.Minute)
        self.symbol = option.Symbol
        self.SetExecution(ImmediateExecutionModel())
        
        option.SetFilter(-1,1,timedelta(30),timedelta(30))
        # use the underlying equity symbol as the benchmark
        self.SetBenchmark(equity.Symbol)

        self.SetRiskManagement(MaximumDrawdownPercentPortfolio(0.25))


    def OnData(self,slice):
        
        optionchain = slice.OptionChains
        for i in slice.OptionChains:
            if i.Key != self.symbol: continue
            chains = i.Value
            contract_list = [x for x in chains]
        # if there is no contracts in this optionchain, pass the instance
        if (slice.OptionChains.Count == 0) or (len(contract_list) == 0): 
            return   
         # if there is no securities in portfolio, trade the options 
        if not self.Portfolio.Invested: 
            self.TradeOptions(optionchain) 
 
    def TradeOptions(self,optionchain):
    
        for i in optionchain:
            if i.Key != self.symbol: continue
            chain = i.Value
            # sorted the optionchain by expiration date and choose the furthest date
            expiry = sorted(chain,key = lambda x: x.Expiry, reverse=True)[0].Expiry
            # filter the call options from the contracts expires on that date
            # 0 for calls, 1 for puts
            call = [i for i in chain if i.Expiry == expiry and i.Right == 1]
            put = [i for i in chain if i.Expiry == expiry and i.Right == 0]
            # sorted the contracts according to their strike prices 
            call_contracts = sorted(call,key = lambda x: x.Strike)
            put_contracts = sorted(call,key = lambda x: x.Strike)
            if len(call_contracts) == 0: continue
            if len(put_contracts) == 0: continue
            # call option contract with lower strike
            self.call_low = call_contracts[0]
            self.put_low = put_contracts[0]
            # call option contract with higher strike
            self.call_high = call_contracts[-1]
            self.put_high = call_contracts[-1]
            
            self.Sell(self.call_high.Symbol, 1)
            self.Sell(self.put_low.Symbol ,1)
    
    def OnOrderEvent(self, orderEvent):
        self.Log(str(orderEvent))